# Lesson 6: Image Histograms
## Biomedical Image Processing - Basic Concepts

### Topics:
- What is an image histogram?
- Histogram analysis and interpretation
- Histogram equalization
- Applications in medical imaging

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

print("Libraries imported successfully!")

## 1. What is a Histogram?

A histogram shows **how many pixels** have each intensity value.

- X-axis: Intensity values (0-255 for 8-bit images)
- Y-axis: Number of pixels with that intensity

In [ ]:
# Create a simple example
simple_img = np.array([
    [50, 50, 100, 100],
    [50, 50, 100, 100],
    [150, 150, 200, 200],
    [150, 150, 200, 200]
], dtype=np.uint8)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Show image
axes[0].imshow(simple_img, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('4x4 Image with 4 intensity values')
for i in range(4):
    for j in range(4):
        axes[0].text(j, i, str(simple_img[i,j]), ha='center', va='center', 
                    color='red' if simple_img[i,j] > 127 else 'yellow', fontsize=12)
axes[0].axis('off')

# Show histogram
hist, bins = np.histogram(simple_img.flatten(), bins=256, range=(0, 256))
axes[1].bar(range(256), hist, width=1, color='steelblue')
axes[1].set_xlim(0, 255)
axes[1].set_xlabel('Intensity Value')
axes[1].set_ylabel('Number of Pixels')
axes[1].set_title('Histogram')

# Annotate peaks
for val in [50, 100, 150, 200]:
    axes[1].annotate(f'{val}: 4 pixels', xy=(val, 4), xytext=(val+10, 4.5),
                    arrowprops=dict(arrowstyle='->', color='red'))

plt.tight_layout()
plt.show()

print("This image has exactly 4 pixels of each intensity: 50, 100, 150, 200")
print("Total pixels: 16")

## 2. Histograms Tell a Story

The shape of a histogram reveals important image characteristics!

In [ ]:
np.random.seed(42)
size = 200

# Create different types of images
# 1. Dark image (low exposure)
dark = np.clip(np.random.normal(50, 20, (size, size)), 0, 255).astype(np.uint8)

# 2. Bright image (high exposure)
bright = np.clip(np.random.normal(200, 20, (size, size)), 0, 255).astype(np.uint8)

# 3. Low contrast
low_contrast = np.clip(np.random.normal(128, 15, (size, size)), 0, 255).astype(np.uint8)

# 4. High contrast
high_contrast = np.clip(np.random.normal(128, 60, (size, size)), 0, 255).astype(np.uint8)

# 5. Bimodal (two distinct regions)
bimodal = np.zeros((size, size), dtype=np.uint8)
bimodal[:size//2, :] = np.clip(np.random.normal(80, 15, (size//2, size)), 0, 255)
bimodal[size//2:, :] = np.clip(np.random.normal(180, 15, (size//2, size)), 0, 255)

# 6. Normal/good exposure
good = np.clip(np.random.normal(128, 40, (size, size)), 0, 255).astype(np.uint8)

images = [
    (dark, 'Dark / Underexposed', 'Most pixels are dark'),
    (bright, 'Bright / Overexposed', 'Most pixels are bright'),
    (low_contrast, 'Low Contrast', 'Narrow histogram'),
    (high_contrast, 'High Contrast', 'Wide histogram'),
    (bimodal, 'Bimodal', 'Two peaks (foreground/background)'),
    (good, 'Well-Exposed', 'Centered, good spread'),
]

fig, axes = plt.subplots(3, 4, figsize=(16, 12))

for i, (img, title, desc) in enumerate(images):
    row = i // 2
    col = (i % 2) * 2
    
    # Image
    axes[row, col].imshow(img, cmap='gray', vmin=0, vmax=255)
    axes[row, col].set_title(title)
    axes[row, col].axis('off')
    
    # Histogram
    hist, _ = np.histogram(img.flatten(), bins=256, range=(0, 256))
    axes[row, col+1].fill_between(range(256), hist, alpha=0.7, color='steelblue')
    axes[row, col+1].set_xlim(0, 255)
    axes[row, col+1].set_title(desc)
    axes[row, col+1].set_xlabel('Intensity')
    axes[row, col+1].set_ylabel('Pixels')

plt.tight_layout()
plt.show()

## 3. Computing Histograms from Scratch

In [ ]:
def compute_histogram(img, num_bins=256):
    """Compute histogram manually."""
    hist = np.zeros(num_bins, dtype=np.int32)
    
    for pixel in img.flatten():
        hist[pixel] += 1
    
    return hist

def compute_cumulative_histogram(hist):
    """Compute cumulative histogram (CDF)."""
    return np.cumsum(hist)

# Test on a simple image
test_img = np.array([[10, 20, 30], [20, 30, 40], [30, 40, 50]], dtype=np.uint8)

hist = compute_histogram(test_img)
cdf = compute_cumulative_histogram(hist)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].imshow(test_img, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('3x3 Test Image')
for i in range(3):
    for j in range(3):
        axes[0].text(j, i, str(test_img[i,j]), ha='center', va='center', color='red')
axes[0].axis('off')

axes[1].bar(range(60), hist[:60], width=1)
axes[1].set_xlabel('Intensity')
axes[1].set_ylabel('Count')
axes[1].set_title('Histogram')

axes[2].plot(range(60), cdf[:60], 'r-', linewidth=2)
axes[2].fill_between(range(60), cdf[:60], alpha=0.3)
axes[2].set_xlabel('Intensity')
axes[2].set_ylabel('Cumulative Count')
axes[2].set_title('Cumulative Distribution Function (CDF)')

plt.tight_layout()
plt.show()

print("Pixel values in image:", sorted(test_img.flatten()))
print(f"Histogram at 10: {hist[10]}, at 20: {hist[20]}, at 30: {hist[30]}, at 40: {hist[40]}, at 50: {hist[50]}")

## 4. Histogram Equalization: The Magic Transformation

Histogram equalization spreads out the intensity values to improve contrast.

**Goal**: Make the histogram as flat (uniform) as possible.

In [ ]:
def histogram_equalization(img):
    """Perform histogram equalization."""
    # Step 1: Compute histogram
    hist = np.zeros(256, dtype=np.int32)
    for pixel in img.flatten():
        hist[pixel] += 1
    
    # Step 2: Compute CDF
    cdf = np.cumsum(hist)
    
    # Step 3: Normalize CDF to 0-255 range
    cdf_normalized = (cdf - cdf.min()) * 255 / (cdf.max() - cdf.min())
    cdf_normalized = cdf_normalized.astype(np.uint8)
    
    # Step 4: Map original values to new values
    equalized = cdf_normalized[img]
    
    return equalized, hist, cdf_normalized

# Apply to low contrast image
low_contrast = np.clip(np.random.normal(128, 20, (200, 200)), 0, 255).astype(np.uint8)
equalized, orig_hist, mapping = histogram_equalization(low_contrast)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Original
axes[0, 0].imshow(low_contrast, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title('Original (Low Contrast)')
axes[0, 0].axis('off')

# Original histogram
orig_h, _ = np.histogram(low_contrast.flatten(), bins=256, range=(0, 256))
axes[0, 1].fill_between(range(256), orig_h, alpha=0.7, color='blue')
axes[0, 1].set_xlim(0, 255)
axes[0, 1].set_title('Original Histogram')
axes[0, 1].set_xlabel('Intensity')

# Mapping function
axes[0, 2].plot(range(256), mapping, 'g-', linewidth=2)
axes[0, 2].plot([0, 255], [0, 255], 'r--', alpha=0.5, label='Identity')
axes[0, 2].set_xlabel('Original Intensity')
axes[0, 2].set_ylabel('New Intensity')
axes[0, 2].set_title('Transformation Function')
axes[0, 2].legend()

# Equalized
axes[1, 0].imshow(equalized, cmap='gray', vmin=0, vmax=255)
axes[1, 0].set_title('Equalized (Higher Contrast)')
axes[1, 0].axis('off')

# Equalized histogram
eq_h, _ = np.histogram(equalized.flatten(), bins=256, range=(0, 256))
axes[1, 1].fill_between(range(256), eq_h, alpha=0.7, color='green')
axes[1, 1].set_xlim(0, 255)
axes[1, 1].set_title('Equalized Histogram')
axes[1, 1].set_xlabel('Intensity')

# Side by side comparison
comparison = np.hstack([low_contrast, equalized])
axes[1, 2].imshow(comparison, cmap='gray', vmin=0, vmax=255)
axes[1, 2].axvline(x=200, color='red', linestyle='--')
axes[1, 2].set_title('Before | After')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("Notice: The equalized histogram is more spread out!")
print("The transformation function (CDF) maps input intensities to output intensities.")

## 5. Histogram Equalization Step by Step

In [ ]:
# Simple example with small image
small_img = np.array([
    [52, 55, 61, 59],
    [62, 59, 55, 55],
    [56, 62, 54, 52],
    [59, 57, 54, 62]
], dtype=np.uint8)

print("Step-by-Step Histogram Equalization")
print("=" * 50)

# Step 1: Count occurrences
unique, counts = np.unique(small_img, return_counts=True)
print("\nStep 1: Count pixel occurrences")
print(f"{'Value':<10} {'Count':<10} {'Probability'}")
total_pixels = small_img.size
for v, c in zip(unique, counts):
    print(f"{v:<10} {c:<10} {c/total_pixels:.4f}")

# Step 2: Compute CDF
print("\nStep 2: Compute Cumulative Distribution (CDF)")
cdf_values = np.cumsum(counts)
print(f"{'Value':<10} {'CDF':<10} {'Normalized CDF (x255)'}")

cdf_min = cdf_values[0]
new_values = {}
for v, cdf in zip(unique, cdf_values):
    # Formula: new_value = ((CDF - CDF_min) / (total - CDF_min)) * 255
    new_val = round(((cdf - cdf_min) / (total_pixels - cdf_min)) * 255)
    new_values[v] = new_val
    print(f"{v:<10} {cdf:<10} {new_val}")

# Step 3: Apply mapping
print("\nStep 3: Apply the transformation")
equalized_small = np.array([[new_values[p] for p in row] for row in small_img], dtype=np.uint8)

print("\nOriginal:")
print(small_img)
print("\nEqualized:")
print(equalized_small)

# Visualize
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(small_img, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')
for i in range(4):
    for j in range(4):
        axes[0].text(j, i, str(small_img[i,j]), ha='center', va='center', color='red')
axes[0].axis('off')

axes[1].bar(unique, counts, width=1)
axes[1].set_title('Original Histogram')
axes[1].set_xlabel('Intensity')

axes[2].imshow(equalized_small, cmap='gray', vmin=0, vmax=255)
axes[2].set_title('Equalized')
for i in range(4):
    for j in range(4):
        axes[2].text(j, i, str(equalized_small[i,j]), ha='center', va='center', color='red')
axes[2].axis('off')

eq_unique, eq_counts = np.unique(equalized_small, return_counts=True)
axes[3].bar(eq_unique, eq_counts, width=10)
axes[3].set_xlim(0, 255)
axes[3].set_title('Equalized Histogram')
axes[3].set_xlabel('Intensity')

plt.tight_layout()
plt.show()

## 6. Medical Imaging Application: X-ray Enhancement

In [ ]:
# Simulate an X-ray image
np.random.seed(42)
size = 200

# Create base X-ray (low contrast, as often acquired)
xray = np.ones((size, size)) * 60  # Dark background

# Add "rib" structures
for y_offset in range(30, 180, 25):
    for x in range(40, 160):
        y = y_offset + int(10 * np.sin(x * 0.05))
        if 0 <= y < size:
            xray[y-2:y+2, x] = 90

# Add "spine"
xray[20:180, 95:105] = 100

# Add "lung" regions (slightly different density)
y, x = np.ogrid[:size, :size]
left_lung = ((x - 60)**2/900 + (y - 100)**2/2500) < 1
right_lung = ((x - 140)**2/900 + (y - 100)**2/2500) < 1
xray[left_lung] = 45
xray[right_lung] = 45

# Add some noise
xray = np.clip(xray + np.random.normal(0, 5, (size, size)), 0, 255).astype(np.uint8)

# Equalize
xray_eq, _, _ = histogram_equalization(xray)

# CLAHE-like local equalization (simplified)
def local_histogram_eq(img, tile_size=50):
    """Simplified adaptive histogram equalization."""
    result = np.zeros_like(img)
    h, w = img.shape
    
    for i in range(0, h, tile_size):
        for j in range(0, w, tile_size):
            tile = img[i:min(i+tile_size, h), j:min(j+tile_size, w)]
            eq_tile, _, _ = histogram_equalization(tile)
            result[i:min(i+tile_size, h), j:min(j+tile_size, w)] = eq_tile
    
    return result

xray_local = local_histogram_eq(xray, 50)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(xray, cmap='gray')
axes[0, 0].set_title('Original X-ray\n(Low Contrast)')
axes[0, 0].axis('off')

axes[0, 1].imshow(xray_eq, cmap='gray')
axes[0, 1].set_title('Global Histogram Equalization')
axes[0, 1].axis('off')

axes[0, 2].imshow(xray_local, cmap='gray')
axes[0, 2].set_title('Local (Adaptive) Equalization')
axes[0, 2].axis('off')

# Histograms
for i, (img, title) in enumerate([(xray, 'Original'), (xray_eq, 'Global EQ'), (xray_local, 'Local EQ')]):
    h, _ = np.histogram(img.flatten(), bins=256, range=(0, 256))
    axes[1, i].fill_between(range(256), h, alpha=0.7)
    axes[1, i].set_xlim(0, 255)
    axes[1, i].set_title(f'{title} Histogram')
    axes[1, i].set_xlabel('Intensity')

plt.tight_layout()
plt.show()

print("Medical Imaging Insight:")
print("- Global equalization can over-enhance noise")
print("- Local/Adaptive methods (CLAHE) often work better for medical images")
print("- CLAHE = Contrast Limited Adaptive Histogram Equalization")

## 7. Histogram Matching (Specification)

What if we want to match the histogram of one image to another?

In [ ]:
def histogram_matching(source, reference):
    """Match source image histogram to reference image histogram."""
    # Get histograms
    src_hist, _ = np.histogram(source.flatten(), bins=256, range=(0, 256))
    ref_hist, _ = np.histogram(reference.flatten(), bins=256, range=(0, 256))
    
    # Compute CDFs
    src_cdf = np.cumsum(src_hist).astype(np.float64)
    src_cdf /= src_cdf[-1]
    
    ref_cdf = np.cumsum(ref_hist).astype(np.float64)
    ref_cdf /= ref_cdf[-1]
    
    # Create mapping
    mapping = np.zeros(256, dtype=np.uint8)
    for i in range(256):
        # Find the reference intensity with closest CDF value
        mapping[i] = np.argmin(np.abs(ref_cdf - src_cdf[i]))
    
    # Apply mapping
    return mapping[source]

# Create source and reference images with different characteristics
np.random.seed(42)
source = np.clip(np.random.normal(80, 20, (150, 150)), 0, 255).astype(np.uint8)  # Dark
reference = np.clip(np.random.normal(150, 40, (150, 150)), 0, 255).astype(np.uint8)  # Bright, high contrast

matched = histogram_matching(source, reference)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(source, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title('Source Image')
axes[0, 0].axis('off')

axes[0, 1].imshow(reference, cmap='gray', vmin=0, vmax=255)
axes[0, 1].set_title('Reference Image\n(Target histogram)')
axes[0, 1].axis('off')

axes[0, 2].imshow(matched, cmap='gray', vmin=0, vmax=255)
axes[0, 2].set_title('Matched Result\n(Source with Reference histogram)')
axes[0, 2].axis('off')

# Histograms
for i, (img, title, color) in enumerate([
    (source, 'Source', 'blue'),
    (reference, 'Reference', 'green'),
    (matched, 'Matched', 'red')
]):
    h, _ = np.histogram(img.flatten(), bins=256, range=(0, 256))
    axes[1, i].fill_between(range(256), h, alpha=0.7, color=color)
    axes[1, i].set_xlim(0, 255)
    axes[1, i].set_title(f'{title} Histogram')
    axes[1, i].set_xlabel('Intensity')

plt.tight_layout()
plt.show()

print("Notice: The matched image has a similar histogram to the reference!")
print("This is useful for standardizing images from different sources.")

## 8. Histogram Statistics

In [ ]:
def compute_histogram_stats(img):
    """Compute various statistics from histogram."""
    pixels = img.flatten()
    
    stats = {
        'Mean': np.mean(pixels),
        'Median': np.median(pixels),
        'Std Dev': np.std(pixels),
        'Min': np.min(pixels),
        'Max': np.max(pixels),
        'Range': np.max(pixels) - np.min(pixels),
        'Variance': np.var(pixels),
    }
    
    # Contrast ratio
    stats['Contrast'] = (stats['Max'] - stats['Min']) / (stats['Max'] + stats['Min'] + 1)
    
    return stats

# Compare statistics of different images
images = {
    'Dark': np.clip(np.random.normal(50, 20, (100, 100)), 0, 255).astype(np.uint8),
    'Bright': np.clip(np.random.normal(200, 20, (100, 100)), 0, 255).astype(np.uint8),
    'Low Contrast': np.clip(np.random.normal(128, 10, (100, 100)), 0, 255).astype(np.uint8),
    'High Contrast': np.clip(np.random.normal(128, 60, (100, 100)), 0, 255).astype(np.uint8),
}

print("HISTOGRAM STATISTICS COMPARISON")
print("=" * 80)
print(f"{'Statistic':<15}", end='')
for name in images.keys():
    print(f"{name:<15}", end='')
print()
print("-" * 80)

all_stats = {name: compute_histogram_stats(img) for name, img in images.items()}

for stat in ['Mean', 'Median', 'Std Dev', 'Min', 'Max', 'Range', 'Contrast']:
    print(f"{stat:<15}", end='')
    for name in images.keys():
        print(f"{all_stats[name][stat]:<15.2f}", end='')
    print()

# Visualize
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for ax, (name, img) in zip(axes, images.items()):
    ax.imshow(img, cmap='gray', vmin=0, vmax=255)
    stats = all_stats[name]
    ax.set_title(f"{name}\nMean={stats['Mean']:.1f}, Std={stats['Std Dev']:.1f}")
    ax.axis('off')

plt.tight_layout()
plt.show()

## 9. Summary

In [ ]:
print("""
SUMMARY: IMAGE HISTOGRAMS
=========================

1. WHAT IS A HISTOGRAM?
   - Distribution of pixel intensities
   - X-axis: Intensity (0-255)
   - Y-axis: Count of pixels

2. HISTOGRAM ANALYSIS
   - Dark image: Histogram shifted left
   - Bright image: Histogram shifted right
   - Low contrast: Narrow histogram
   - High contrast: Wide histogram
   - Bimodal: Two peaks (foreground/background)

3. HISTOGRAM EQUALIZATION
   - Uses CDF to redistribute intensities
   - Makes histogram more uniform
   - Improves contrast automatically
   - Formula: new = (CDF - CDF_min) / (N - CDF_min) * 255

4. TYPES OF EQUALIZATION
   - Global: Whole image at once
   - Local/Adaptive (CLAHE): Tile by tile
   - CLAHE is better for medical images

5. HISTOGRAM MATCHING
   - Match one image's histogram to another
   - Useful for standardization

6. STATISTICS FROM HISTOGRAMS
   - Mean, median, standard deviation
   - Min, max, range
   - Contrast ratio

Histograms are the "fingerprint" of an image!
""")